In [14]:
import pandas as pd
import ast

data = pd.read_csv('paired_data.csv')
data["overview_embedding"] = data["overview_embedding"].apply(ast.literal_eval)

In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from joblib import dump

# Convert embedding column from list to numpy array
data["overview_embedding"] = data["overview_embedding"].apply(np.array)

# Split the data into features (X) and target (y)
X = np.vstack(data["overview_embedding"].values)
y = data['average_rating']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the neural network architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=12, batch_size=8, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

# Save the model
dump(scaler, 'scaler.joblib')
model.save('movie_rating_prediction_model.keras')

/home/matt/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 1.3380 - val_loss: 0.6068
Epoch 2/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.3891 - val_loss: 0.5530
Epoch 3/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.2811 - val_loss: 0.5480
Epoch 4/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.2025 - val_loss: 0.5738
Epoch 5/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.1704 - val_loss: 0.5470
Epoch 6/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.1524 - val_loss: 0.5645
Epoch 7/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.1324 - val_loss: 0.5859
Epoch 8/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0988 - val_loss: 0.5739
Epoch 9/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0918 - val_loss: 0.5724
Epoch 10/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0921 - val_loss: 0.5925
Epoch 11/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0859 - val_loss: 0.5550
Epoch 12/12
596/596 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step

In [55]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model
transformer = SentenceTransformer('all-MiniLM-L6-v2')

overview: str = input("Give your movie overview: ")
print(overview)

# Now you can use the loaded scaler to transform new data
scaled_embed = scaler.transform(np.array(transformer.encode(overview)).reshape(1, -1))

predicted_rating = model.predict(scaled_embed)

print(f"prediction: {predicted_rating}")


/home/matt/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Imprisoned in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
prediction: [[3.5972214]]


In [ ]:
from sklearn.preprocessing import StandardScaler
from joblib import load
import numpy as np

overview: str = input("Give your movie overview: ")
print(overview)

# Assuming scaler is your StandardScaler object
scaler = StandardScaler()

# Later, to load the scaler from the file
scaler = load('scaler.joblib')

# Now you can use the loaded scaler to transform new data
scaled_embed = scaler.transform(np.array(transformer.encode(overview)).reshape(1, -1))

predicted_rating = model.predict(scaled_embed)

print(f"prediction: {predicted_rating}")

The filmmaking team behind the hits "Scary Movie," "Date Movie," "Epic Movie" and "Meet The Spartans" this time puts its unique, inimitable stamp on one of the biggest and most bloated movie genres of all time -- the disaster film.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: [[3.3559082]]
